In [89]:
import numpy as np
import matplotlib.pyplot as plt

# Define WLS plate properties
n_wls_plate = 1.5  # refractive index of the WLS plate
plate_width = 0.3  # width of the WLS plate
plate_thickness = 0.005  # thickness of the WLS plate
plate_length = 0.3  # length of the WLS plate

# Define ellipsoid properties for the PMT
pmt_radius_x = 0.0762 / 2  # radius of the PMT along x-axis
pmt_radius_y = 0.0762 / 2  # radius of the PMT along y-axis
pmt_radius_z = 40e-3 / 2  # radius of the PMT along z-axis

# Initialize ray parameters
initial_position = np.array([0, 0, 0])  # starting point

# Randomize the initial direction
phi = 2 * np.pi * np.random.rand()  # azimuthal angle
theta = np.arccos(2 * np.random.rand() - 1)  # polar angle
initial_direction = np.array([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])

# Simulate ray trajectory
ray_positions = [initial_position]
ray_directions = [initial_direction]

while True:
    # Calculate the refracted direction inside the WLS plate
    refracted_direction = np.arcsin(np.sin(np.arctan2(np.linalg.norm(ray_directions[-1][:2]), ray_directions[-1][2])) / n_wls_plate)
    
    # Update ray position inside the plate
    new_position = ray_positions[-1] + refracted_direction * ray_directions[-1] / np.linalg.norm(ray_directions[-1])
    
    # Check if the ray hits the PMT
    if (
        (new_position[0] / pmt_radius_x)**2 +
        (new_position[1] / pmt_radius_y)**2 +
        (new_position[2] / pmt_radius_z)**2 <= 1
    ):
        ray_positions.append(new_position)
        break
    
    # Check if the ray hits the upper boundary of the WLS plate
    if new_position[2] >= plate_thickness:
        break
    
    # Check if the ray hits the WLS plate
    if (
        -plate_length / 2 <= new_position[0] <= plate_length / 2 and
        -plate_width / 2 <= new_position[1] <= plate_width / 2 and
        0 <= new_position[2] <= plate_thickness
    ):
        # Reflect the ray at the plate boundary
        reflected_direction = ray_directions[-1] - 2 * np.dot(ray_directions[-1], refracted_direction) * refracted_direction
        ray_positions.append(new_position)
        ray_directions.append(reflected_direction)
    else:
        ray_positions.append(new_position)
        ray_directions.append(ray_directions[-1])

# Extract X, Y, Z coordinates for plotting
x_coords, y_coords, z_coords = zip(*ray_positions)

# Plot the trajectory along each axis
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(x_coords, y_coords, '-o')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('XY Projection')

plt.subplot(1, 3, 2)
plt.plot(x_coords, z_coords, '-o')
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
plt.title('XZ Projection')

plt.subplot(1, 3, 3)
plt.plot(y_coords, z_coords, '-o')
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')
plt.title('YZ Projection')

# Plot the ellipsoidal PMT in the center
ellipsoid = plt.Circle((0, 0), pmt_radius_x, color='red', alpha=0.5)
plt.gca().add_patch(ellipsoid)

plt.tight_layout()
plt.show()


KeyboardInterrupt: 